In [2]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
from sqlalchemy import create_engine
from flask_cors import CORS  # Import CORS

In [3]:
app = Flask(__name__)
CORS(app)
# Thông tin kết nối cơ sở dữ liệu
db_user = 'root'
db_password = '1234'
db_host = 'localhost'
db_port = '3306'
db_name = 'doan_travel'

In [4]:
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

In [12]:
# Truy vấn dữ liệu từ các bảng
customer_query = "SELECT id AS customer_id, name, email FROM customer"
tour_query = "SELECT id AS tour_id, title FROM tour"
tour_queryall="select *from tour"
review_query = "SELECT customer_id, tour_id, rating FROM review"

In [13]:
# Đọc dữ liệu vào DataFrame
customer_df = pd.read_sql(customer_query, engine)
tour_df = pd.read_sql(tour_query, engine)
review_df = pd.read_sql(review_query, engine)
tour_all=pd.read_sql(tour_queryall,engine)

In [7]:
# Đổi tên cột 'id' thành 'customer_id' trong customer_df và 'tour_id' trong tour_df
customer_df.rename(columns={'id': 'customer_id'}, inplace=True)
tour_df.rename(columns={'id': 'tour_id'}, inplace=True)

In [24]:
review_df

,customer_id,tour_id,rating
0,1,46,5.0
1,1,678,5.0
2,1,1189,4.0
3,1,1261,4.0
4,1,2216,0.0
...,...,...,...
1500,100,4624,5.0
1501,100,5843,5.0
1502,100,6149,1.0
1503,100,7411,2.0


In [26]:
customer_df

,customer_id,name,email
0,1,None,vinhbui21@gmail.com
1,2,None,vinhhoc21@gmail.com
2,3,None,string
3,4,None,vinhhoc22@gmail.com
4,5,None,1@email.com
...,...,...,...
96,97,None,zherrera@example.net
97,98,None,ddean@example.org
98,99,None,luke73@example.org
99,100,None,plong@example.net


In [28]:
tour_all

,id,is_abroad,end_date,start_date,description,price_currency,schedule,title,img_url,departure,destination,price,img_url2,img_url3
0,1,0,2024-12-05,2024-12-01,Vịnh Hạ Long là một trong những kỳ quan thiên ...,VND,"Ngày 1, du khách sẽ khởi hành từ Hà Nội vào sá...",Tham quan Vịnh Hạ Long,https://golden-lotus-hotel.s3.ap-southeast-1.a...,Hà Nội,Quảng Ninh,1000000.0,None,None
1,2,0,2024-12-14,2024-12-01,Đà Nẵng là một trong những thành phố biển nổi ...,VND,"Ngày 1, du khách sẽ khởi hành từ sáng sớm, đến...",Tham quan Đà Nẵng,https://vcdn1-dulich.vnecdn.net/2022/06/03/cau...,Hà Nội,Đà Nẵng,2000000.0,None,None
2,3,0,2024-12-14,2024-12-01,Thanh Hóa là một tỉnh ven biển nằm ở miền Bắc ...,VND,"Ngày 1, du khách sẽ khởi hành từ sáng sớm và đ...",Khám phá Sầm Sơn,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",Hà Nội,Thanh Hóa,10000000.0,None,None
3,4,0,2024-12-14,2024-12-01,Phố cổ là một khu vực đặc trưng của nhiều thàn...,VND,"Ngày 1, du khách sẽ khởi hành vào buổi sáng và...",Khám phá Phố cổ Hà Nội,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",Hà Nội,Hà Nội,30000000.0,None,None
4,5,0,2024-12-05,2024-12-01,Quy Nhơn là thành phố ven biển thuộc tỉnh Bình...,VND,Ngày 1: Tham quan thành phố Quy Nhơn. Ngày 2: ...,Khám Phá Quy Nhơn,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",Hà Nội,Quy Nhơn,3500000.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,9996,1,2024-08-17,2023-12-29,Tour khám phá Matthewshire với các địa điểm nổ...,VND,Ngày 1: About various so still strategy recogn...,Khám phá Port Luis - Tour 9996,https://placekitten.com/725/469,Port Cameron,Jacksonside,5242700.0,https://picsum.photos/85/207,https://placekitten.com/762/92
9992,9997,1,2024-04-10,2023-06-25,Tour khám phá Darrenchester với các địa điểm n...,VND,Ngày 1: Statement event son least. | Ngày 2: S...,Khám phá Port Eric - Tour 9997,https://placekitten.com/152/585,North Hannahview,Lukeberg,7607900.0,https://placekitten.com/152/484,https://picsum.photos/701/405
9993,9998,1,2024-02-24,2023-03-08,Tour khám phá Crystalmouth với các địa điểm nổ...,VND,Ngày 1: Painting recognize rise money. | Ngày ...,Khám phá Jacquelinebury - Tour 9998,https://picsum.photos/769/328,North Rebecca,New Amanda,3311300.0,https://placekitten.com/626/726,https://dummyimage.com/53x943
9994,9999,1,2024-04-23,2023-01-14,Tour khám phá Katieburgh với các địa điểm nổi ...,VND,Ngày 1: Beautiful statement think get mind aut...,Khám phá Hillville - Tour 9999,https://dummyimage.com/527x141,Travistown,East Jennaland,1732900.0,https://picsum.photos/655/233,https://picsum.photos/952/496


In [15]:
# Mở rộng bảng review với thông tin tour và khách hàng
merged_df = pd.merge(review_df, customer_df[['customer_id']], on='customer_id')
merged_df = pd.merge(merged_df, tour_df[['tour_id', 'title']], on='tour_id')
merged_df = merged_df[merged_df['rating'] > 0]

In [25]:
merged_df

,customer_id,tour_id,rating,title
0,1,46,5.0,Khám phá Gabrielview - Tour 46
1,1,678,5.0,Khám phá Andersonmouth - Tour 678
2,1,1189,4.0,Khám phá Staceyside - Tour 1189
3,1,1261,4.0,Khám phá Williamsonland - Tour 1261
5,1,2310,1.0,Khám phá Melissahaven - Tour 2310
...,...,...,...,...
1500,100,4624,5.0,Khám phá Port Amy - Tour 4624
1501,100,5843,5.0,Khám phá Gregorychester - Tour 5843
1502,100,6149,1.0,Khám phá Reedburgh - Tour 6149
1503,100,7411,2.0,Khám phá East Krista - Tour 7411


In [17]:
pivot_df = merged_df.pivot(index='customer_id', columns='tour_id', values='rating').fillna(0)

In [19]:
X = pivot_df.values

In [21]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(X)


NearestNeighbors(algorithm='brute', metric='cosine')

In [22]:
# API để gợi ý tour cho khách hàng
@app.route('/recommend', methods=['GET'])
def recommend():
    customer_id = request.args.get('customer_id', type=int)
    
    # Kiểm tra xem khách hàng có tồn tại không
    if customer_id not in pivot_df.index:
        return jsonify({"error": "Customer not found"}), 404
    
    # Tìm 5 khách hàng tương tự
    customer_index = pivot_df.index.get_loc(customer_id)
    distances, indices = model.kneighbors([X[customer_index]], n_neighbors=6)  # 6 vì khách hàng này sẽ là 1 trong số đó

    # Lấy danh sách các tour đã được đánh giá của các khách hàng tương tự
    similar_customers = pivot_df.index[indices.flatten()[1:]]  # Bỏ qua chính khách hàng này
    
    # Tạo danh sách các tour đã được đánh giá của các khách hàng tương tự
    recommended_tours = set()  # Dùng set để loại bỏ các tour bị trùng
    for similar_customer in similar_customers:
        customer_ratings = pivot_df.loc[similar_customer]
        rated_tours = customer_ratings[customer_ratings > 0].index.tolist()  # Các tour có rating > 0
        recommended_tours.update(rated_tours)  # Thêm các tour vào set

    # Chuyển sang danh sách và lấy ngẫu nhiên 5 tour
    recommended_tours_list = list(recommended_tours)
    random.shuffle(recommended_tours_list)  # Trộn ngẫu nhiên các tour
    recommended_tours_list = recommended_tours_list[:7]  
    return jsonify({"recommended_tours": recommended_tours_list})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\THINKPAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
